## Create _S.salar_ UniProt annotations file for Shelly

See this [GitHub Issue](https://github.com/RobertsLab/resources/issues/1220)

This notebook relies on [GFFutils](https://gffutils.readthedocs.io/en/v0.12.0/index.html) to be installed and available in your `$PATH`.

### List computer specs

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Tue 01 Jun 2021 07:37:57 AM PDT
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.2 LTS
Release:	20.04
Codename:	focal

------------
HOSTNAME: 
computer

------------
Computer Specs:

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   45 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              1
Core(s) per socket:              1
Socket(s):                       8
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           165
Model name:                      Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz
Stepping:                        2
CPU MHz:                         2400.000
BogoMIPS:                        4800.00
Hypervisor vendor:               VMware
Virtualization type:    

No LSB modules are available.


### Set variables
- `%env` indicates a bash variable; without `%env` is Python variable

In [2]:
# Set directories, input/output files
%env data_dir=/home/samb/data/S_salar/genomes
%env analysis_dir=/home/samb/analyses/20210601_ssal_gff-annotations
analysis_dir="/home/samb/analyses/20210601_ssal_gff-annotations"
# Input GFF
%env orig_gff=GCF_000233375.1_ICSASG_v2_genomic.gff
%env orig_gff_url=https://gannet.fish.washington.edu/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq

# UniProt batch output
%env perl_output=20210601_ssal_uniprot_batch_results.txt

# GTF extractor output
%env gtf_extractor_output=20210601_ssal_chrom-start-end-Dbxref.csv

# Gene name list for UniProt batch submission
%env gene_list=20210601_ssal_gene-list.txt

# Parsed UniProt
%env parsed_uniprot=20210601_ssal_accession-gene_id-gene-gene_description-go_ids.csv

# Final output
%env joined_output=20210601_ssal_chrom-gene_id_start-end-acc-gene-gene_description-go_ids.csv

env: data_dir=/home/samb/data/S_salar/genomes
env: analysis_dir=/home/samb/analyses/20210601_ssal_gff-annotations
env: orig_gff=GCF_000233375.1_ICSASG_v2_genomic.gff
env: orig_gff_url=https://gannet.fish.washington.edu/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq
env: perl_output=20210601_ssal_uniprot_batch_results.txt
env: gtf_extractor_output=20210601_ssal_chrom-start-end-Dbxref.csv
env: gene_list=20210601_ssal_gene-list.txt
env: parsed_uniprot=20210601_ssal_accession-gene_id-gene-gene_description-go_ids.csv
env: joined_output=20210601_ssal_chrom-gene_id_start-end-acc-gene-gene_description-go_ids.csv


### Download GFF

In [3]:
%%bash
cd "${data_dir}"

# Download with wget. Use --no-check-certificate to avoid issues with Gannet certificate
# Use --quiet option to prevent wget output from printing too many lines to notebook
wget --no-check-certificate --quiet ${orig_gff_url}/${orig_gff}

ls -ltrh "${orig_gff}"

-rw-rw-r-- 1 samb samb 828M Sep 30  2020 GCF_000233375.1_ICSASG_v2_genomic.gff


### Examine GFF

In [4]:
%%bash
head -n 20 "${data_dir}"/"${orig_gff}"

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build ICSASG_v2
#!genome-build-accession NCBI_Assembly:GCF_000233375.1
#!annotation-source NCBI Salmo salar Annotation Release 100
##sequence-region NC_027300.1 1 159038749
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=8030
NC_027300.1	RefSeq	region	1	159038749	.	+	.	ID=NC_027300.1:1..159038749;Dbxref=taxon:8030;Name=ssa01;breed=double haploid;chromosome=ssa01;dev-stage=adult;gbkey=Src;genome=chromosome;isolate=Sally;mol_type=genomic DNA;sex=female;tissue-type=muscle
NC_027300.1	Gnomon	gene	5501	62139	.	-	.	ID=gene-LOC106560212;Dbxref=GeneID:106560212;Name=LOC106560212;gbkey=Gene;gene=LOC106560212;gene_biotype=protein_coding
NC_027300.1	Gnomon	mRNA	5501	62139	.	-	.	ID=rna-XM_014160784.1;Parent=gene-LOC106560212;Dbxref=GeneID:106560212,Genbank:XM_014160784.1;Name=XM_014160784.1;gbkey=mRNA;gene=LOC106560212;model_evidence=Supporting evidence includes similarity to: 99%25 coverage of the 

### Use GFFutils to extract gene features

In [7]:
%%bash
# Extract just gene features
# Extract chromosome name, start, end, and Dbxref fields
# Dbxref is the NCBI gene name, in this particular instance
# Specify input as GFF
# Use awk to format as comma-delimited output to help with downstream parsing/joining
time \
gtf_extract \
--feature gene \
--fields=chrom,start,end,Dbxref \
--gff ${data_dir}/${orig_gff} \
| awk 'BEGIN { OFS = ","; FS="[\t:]"} {print $1, $2, $3, $5}' \
> ${analysis_dir}/${gtf_extractor_output}


real	2m26.807s
user	2m26.279s
sys	0m0.468s


#### Check results

In [8]:
%%bash
cd "${analysis_dir}"
ls -ltrh ${gtf_extractor_output}

echo ""

head ${gtf_extractor_output}

-rw-rw-r-- 1 samb samb 2.9M Jun  1 09:49 20210601_ssal_chrom-start-end-Dbxref.csv

NC_027300.1,5501,62139,106560212
NC_027300.1,160437,198815,106607996
NC_027300.1,228330,231471,106601976
NC_027300.1,296031,297111,106560213
NC_027300.1,306942,310878,106566220
NC_027300.1,331369,346454,106571988
NC_027300.1,355675,362950,106578259
NC_027300.1,401623,416794,106583877
NC_027300.1,431662,432555,106589664
NC_027300.1,449112,490663,106596642


#### Confirm that GFFutils output seem okay

In [10]:
%%bash
# Count gene features via GFFutils
echo "GFFutils number of extracted genes:"
gtf_extract -f gene --fields=Dbxref --gff ${data_dir}/${orig_gff} | wc -l

echo ""

# Count gene features via awk
echo "awk number of extracted genes:"
awk '$3 == "gene" { print $0 }' ${data_dir}/${orig_gff} | wc -l


GFFutils number of extracted genes:
79030

awk number of extracted genes:
79030


### Extract gene ids for batch submission to UniProt

In [11]:
%%bash
cd "${analysis_dir}"
awk -F"," '{print $4}' "${gtf_extractor_output}" > "${gene_list}"

#### Check gene list

In [12]:
%%bash
cd "${analysis_dir}"
head "${gene_list}"

106560212
106607996
106601976
106560213
106566220
106571988
106578259
106583877
106589664
106596642


### Batch submission to UniProt

Perl script obtained from UniProt: https://www.uniprot.org/help/api_batch_retrieval

Modified to map NCIB gene ID to UniProt accession.

In [13]:
%%bash
# Print the script for viewing
cat /home/samb/programs/uniprot_mapping.pl

use strict;
use warnings;
use LWP::UserAgent;

my $list = $ARGV[0]; # File containg list of UniProt identifiers.

my $base = 'https://www.uniprot.org';
my $tool = 'uploadlists';

my $contact = 'samwhite@uw.edu'; # Please set a contact email address here to help us debug in case of problems (see https://www.uniprot.org/help/privacy).
my $agent = LWP::UserAgent->new(agent => "libwww-perl $contact");
push @{$agent->requests_redirectable}, 'POST';

my $response = $agent->post("$base/$tool/",
                            [ 'file' => [$list],
                              'format' => 'txt',
                              'from' => 'P_ENTREZGENEID',
                              'to' => 'ACC',
                            ],
                            'Content_Type' => 'form-data');

while (my $wait = $response->header('Retry-After')) {
  print STDERR "Waiting ($wait)...\n";
  sleep $wait;
  $response = $agent->get($response->base);
}

$response->is_success ?
  print $response->content :
  die 

In [16]:
%%bash
cd "${analysis_dir}"
time \
perl /home/samb/programs/uniprot_mapping.pl "${gene_list}" > "${perl_output}"
ls -ltrh

echo ""
echo ""
echo "--------------------------------------------------"
echo ""
echo "Line count:"
wc -l "${perl_output}"

total 354M
-rw-rw-r-- 1 samb samb 2.9M Jun  1 09:49 20210601_ssal_chrom-start-end-Dbxref.csv
-rw-rw-r-- 1 samb samb 772K Jun  1 09:58 20210601_ssal_gene-list.txt
-rw-rw-r-- 1 samb samb 350M Jun  1 10:04 20210601_ssal_uniprot_batch_results.txt


--------------------------------------------------

Line count:
7273462 20210601_ssal_uniprot_batch_results.txt



real	3m32.207s
user	0m3.099s
sys	0m9.446s


#### Check mapping output

In [17]:
%%bash
cd "${analysis_dir}"
head -n 30 "${perl_output}"

ID   A0A1S3NLH1_SALSA        Unreviewed;       218 AA.
AC   A0A1S3NLH1;
DT   12-APR-2017, integrated into UniProtKB/TrEMBL.
DT   12-APR-2017, sequence version 1.
DT   07-APR-2021, entry version 14.
DE   SubName: Full=fibroblast growth factor receptor 3-like {ECO:0000313|RefSeq:XP_014016259.1};
GN   Name=LOC106560212 {ECO:0000313|RefSeq:XP_014016259.1};
OS   Salmo salar (Atlantic salmon).
OC   Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
OC   Actinopterygii; Neopterygii; Teleostei; Protacanthopterygii; Salmoniformes;
OC   Salmonidae; Salmoninae; Salmo.
OX   NCBI_TaxID=8030 {ECO:0000313|Proteomes:UP000087266, ECO:0000313|RefSeq:XP_014016259.1};
RN   [1] {ECO:0000313|RefSeq:XP_014016259.1}
RP   IDENTIFICATION.
RC   TISSUE=Muscle {ECO:0000313|RefSeq:XP_014016259.1};
RG   RefSeq;
RL   Submitted (OCT-2020) to UniProtKB.
CC   ---------------------------------------------------------------------------
CC   Copyrighted by the UniProt Consortium, see https://www.uniprot.org/

### Parse out the stuff we want:

- UniProt accession

- Gene ID (NCBI gene ID)

- Gene name/abbraviation

- Gene description

- GO terms

In [ ]:
%%bash
cd "${analysis_dir}"
time \
while read -r line
do
  # Get record line descriptor
  descriptor=$(echo "${line}" | awk '{print $1}' )

  # Capture second field for evaluation
  go_line=$(echo "${line}" | awk '{print $2}')

  # Append GO IDs to array
  if [[ "${go_line}" == "GO;" ]]; then
    go_id=$(echo "${line}" | awk '{print $3}')
    go_ids_array+=("${go_id}")
  elif [[ "${go_line}" == "GeneID;" ]]; then
    # Uses sed to strip trailing semi-colon
    gene_id=$(echo "${line}" | awk '{print $3}' | sed 's/;$//')
  fi

  # Get gene description
  if [[ "${descriptor}"  == "DE" ]] && [[ "${go_line}"  == "SubName:" ]]; then
    # Uses sed to strip trailing spaces at end of line and remove commas
    gene_description=$(echo "${line}" | awk -F"[={]" '{print $2}' | sed 's/[[:blank:]]*$//' | sed 's/,//g')

  # Get gene name
  elif [[ "${descriptor}"  == "GN"  ]] && [[ $(echo "${line}" | awk -F "=" '{print $1}') == "GN   Name" ]]; then
    # Uses sed to strip trailing spaces at end of line
    gene=$(echo "${line}" | awk -F'Name=|{' '{print $2}' | sed 's/[[:blank:]]*$//')

  # Get UniProt accession
  elif [[ "${descriptor}"  == "AC" ]]; then
    # Uses sed to strip trailing semi-colon
    accession=$(echo "${line}" | awk '{print $2}' | sed 's/;$//')

  # Identify beginning on new record
  elif [[ "${descriptor}"  == "//" ]]; then

    # Prints other comma-separated variables, then GOID1;GOID2;GOIDn
    # IFS prevents spaces from being added between GO IDs
    # sed removes ";" after final GO ID
    (IFS=; printf "%s,%s,%s,%s,%s\n" "${accession}" "${gene_id}" "${gene}" "${gene_description}" "${go_ids_array[*]}" | sed 's/;$//')

    # Re-initialize variables
    accession=""  
    descriptor=""
    gene=""
    gene_description
    gene_id=""
    go_id=""
    go_ids_array=()
  fi


done < "${perl_output}" >> "${parsed_uniprot}"

#### Check parsed file

In [ ]:
%%bash
head "${parsed_uniprot}"

echo ""
echo ""
echo "--------------------------------------------------"
echo ""
echo "Line count:"
wc -l "${perl_output}"

### Join parsed UniProt info with chromosome names

In [ ]:
%%bash
join \
-t "," \
-1 4 \
-2 2 \
<(sort -t "," -k 4,4 "${gtf_extractor_output}")  \
<(sort -t "," -k2,2 "${parsed_uniprot}") \
| awk 'BEGIN {FS=","; OFS="\t"} {print $2, $1, $3, $4, $5, $6, $7, $8}' \
> "${joined_output}"